In [3]:
import pandas as pd
import pickle
import numpy as np
import networkx as nx
import re
import time

In [4]:
df_ivs = pickle.load(open('data/ivs_exploded_100.p', 'rb'))
df_ivs_old = pickle.load(open('data/ivs_exploded_100_old.p', 'rb'))
routes_dests = pickle.load(open('data/users_ship_specific_routes.p', 'rb'))
df_ships = pd.read_excel('data/ship_types.xlsx')

G = pickle.load(open("data/network.p", "rb"))
paths = pickle.load(open("data/paths.p", "rb"))
df_random = pickle.load(open("data/df_random.p", "rb"))
df_abm = pickle.load(open("data/df_abm.p", "rb"))
non_zero_flows = pickle.load(open("data/non_zero_flows.p", "rb"))
feasible_combinations = pickle.load(open('data/feasible_comb.p', 'rb'))
optimal_flows = pickle.load(open('data/optimal_flows.p', 'rb'))

In [5]:
df_abm.loc[df_abm.name]

,id,source,target,length_m,X,Y,name,charging_stations,model_type,range,capacity
0,1000,113,8867600,20435.805397,5.269204,53.215456,"(113, 8867600)",NaN,link,100000,2500
1,1001,113,8865605,20435.805397,5.151546,53.267585,"(113, 8865605)",NaN,link,100000,2500
2,1002,8865756,30986055,2481.139176,5.412350,53.178500,"(8865756, 30986055)",NaN,link,100000,2500
3,1003,8863630,8863818,7765.172161,4.835893,51.681540,"(8863630, 8863818)",NaN,link,100000,2500
4,1004,8863630,8868359,1034.856186,4.830372,51.711997,"(8863630, 8868359)",NaN,link,100000,2500
...,...,...,...,...,...,...,...,...,...,...,...
211,1211,NaN,NaN,NaN,5.536470,51.816354,8860920,0.0,intermediate_node,100000,2500
212,1212,NaN,NaN,NaN,4.846061,51.651560,8863818,1.0,harbour_with_charging,100000,2500
213,1213,NaN,NaN,NaN,3.705824,51.451397,8864111,0.0,harbour,100000,2500
214,1214,NaN,NaN,NaN,4.961690,51.824861,8864151,0.0,intermediate_node,100000,2500


In [8]:
o = {}
for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a] = {'combinations':[], 'flows':[]}

for key, item in optimal_flows.items():
    if item != 0:
        a = (re.sub('''["'_']''', "", key[16:35]).split(','))
        a = tuple([a[0], a[1], int(a[2])])
        o[a]['combinations'].append(key[40:-4].split("',_'"))
        o[a]['flows'].append(item)
        print(a)
        print(key[40:-4].split("',_'"))


('NLABL', 'NLRTM', 0)
['8866686']
('NLAER', 'NLLEY', 0)
['8863709']
('NLAER', 'NLLEY', 0)
['8864005']
('NLAER', 'NLRTM', 0)
['8863360', '886400']
('NLAER', 'NLSWI', 0)
['8864005', '886826']
('NLALK', 'NLIJM', 0)
['8863260']
('NLALK', 'NLIJM', 0)
['8866889']
('NLAMS', 'NLAER', 0)
['8866763', '886400']
('NLAMS', 'NLTIE', 0)
['8864837', '2263814']
('NLAMS', 'NLUTC', 1)
['8864837']
('NLAMS', 'NLUTC', 1)
['8867240', '886676']
('NLAMS', 'NLWMO', 0)
['8866889']
('NLAMS', 'NLZAA', 0)
['8866889']
('NLDOR', 'NLRTM', 0)
['8862663']
('NLDOR', 'NLRTM', 1)
['8862663']
('NLDRO', 'NLKAM', 0)
['8868191']
('NLFAR', 'NLEEM', 0)
['8863614']
('NLGOR', 'NLRTM', 0)
['8866686']
('NLHAR', 'NLWTE', 0)
['113']
('NLHAR', 'NLWTE', 0)
['8865756', '886560']
('NLHTB', 'NLBON', 0)
['8865990', '886820']
('NLHTB', 'NLBZM', 0)
['112', '886677']
('NLIJM', 'NLRTM', 1)
['8863360', '886326']
('NLKGZ', 'NLAMS', 0)
['8863260']
('NLKGZ', 'NLZAA', 0)
['8866889']
('NLLEY', 'NLAMS', 0)
['8863709', '886676']
('NLMOE', 'NLOOS', 0)
[

In [9]:
o

{('NLABL', 'NLRTM', 0): {'combinations': [['8866686']], 'flows': [1.0]},
 ('NLAER', 'NLLEY', 0): {'combinations': [['8863709'], ['8864005']],
  'flows': [0.40542137, 0.59457863]},
 ('NLAER', 'NLRTM', 0): {'combinations': [['8863360', '886400']],
  'flows': [0.21725455]},
 ('NLAER', 'NLSWI', 0): {'combinations': [['8864005', '886826']],
  'flows': [1.0]},
 ('NLALK', 'NLIJM', 0): {'combinations': [['8863260'], ['8866889']],
  'flows': [0.56482298, 0.43517702]},
 ('NLAMS', 'NLAER', 0): {'combinations': [['8866763', '886400']],
  'flows': [0.5437747]},
 ('NLAMS', 'NLTIE', 0): {'combinations': [['8864837', '2263814']],
  'flows': [0.21140541]},
 ('NLAMS', 'NLUTC', 1): {'combinations': [['8864837'], ['8867240', '886676']],
  'flows': [0.96552342, 0.034476582]},
 ('NLAMS', 'NLWMO', 0): {'combinations': [['8866889']], 'flows': [1.0]},
 ('NLAMS', 'NLZAA', 0): {'combinations': [['8866889']], 'flows': [1.0]},
 ('NLDOR', 'NLRTM', 0): {'combinations': [['8862663']], 'flows': [1.0]},
 ('NLDOR', 'NLR

In [55]:
a = o[('NLAMS', 'NLUTC', 1)]
combi = np.random.choice(a=np.arange(len(a)), size=1, replace=False, p=[0.5,0.5])

In [61]:
a = [1,2,3]
a = [i/sum(a) for i in a]

In [62]:
a

[0.16666666666666666, 0.3333333333333333, 0.5]

In [98]:
df_abm.loc[df_abm.name == '8864005']

,id,source,target,length_m,X,Y,name,charging_stations,model_type,range,capacity
143,1143,NaN,NaN,NaN,5.239614,52.403333,8864005,5.0,harbour_with_charging,100000,5


In [81]:
updated_feasible_combinations = {}
for key, combinations in feasible_combinations.items():
    updated_feasible_combinations[key] = []
    if len(combinations) > 1:
        for combi in combinations:
            # print(combi)
            feasible = True
            for node in combi:
                if not node in charging_points:
                    feasible = False
                    break
            if feasible:
                updated_feasible_combinations[key].append(combi)
    else:
        feasible = True
        for node in combinations:
            if not node in charging_points:
                feasible = False
                break
        if feasible:
            updated_feasible_combinations[key].append(combinations)



In [28]:
a = [1]
a[0]

1

In [4]:
# extract data
df_links = nx.to_pandas_edgelist(G)
df_nodes = pd.DataFrame.from_dict(dict(G.nodes(data=True)), orient='index')

In [7]:
for _, row in df_abm.iterrows():  # index, row in ...
    # create agents according to model_type

    model_type = row['model_type']
    agent = None

    name = row['name']
    if pd.isna(name):
        name = ""
        print("error, nameless entry")
    else:
        name = name
    break

In [10]:
a =[1,3,34,6]
a[::-1]

[6, 34, 3, 1]

In [36]:
df_random.trip_count.iloc[0]

54

In [3]:
%%time
type_list = list(df_random.iloc[:, 4:-2])
for hour in range(24):
    df_1 = df_random.loc[(df_random.hour == hour)]
    for timestep in range(60):
        for harbour in list(set(list(df_1.origin.unique()) + list(df_1.destination.unique()))):
            a = df_1.loc[(df_1.origin==harbour)|(df_1.destination == harbour)]
            for i,j in enumerate(a.index):
                # chance that a vessel is generated is equal to 1/2 (either spawn at origin or dest) * trip_count/365
                # (because trip_count is yearly) *(time_step/hours)
                if 0.5 * (df_1.trip_count[j]/365) * (1/60) >= np.random.random():
                    # determine vessel type
                    prob = list(a.iloc[i, 4:-2].values / a.iloc[i, 4:-2].sum())
                    to_pick = type_list
                    ship_type = np.random.choice(a=to_pick, size=1, replace=True, p=prob)
                    # randomly draw ship types for each OD pair
                    print(ship_type, "Vessel departed at", df_1.origin[j], hour, ':', timestep, "heading to", df_1.destination[j], "via route", df_1.key[j])


['M10'] Vessel departed at NLTNZ 3 : 14 heading to NLVLI via route ('NLTNZ', 'NLVLI', 0)
['M8'] Vessel departed at NLOOS 3 : 44 heading to NLTLB via route ('NLOOS', 'NLTLB', 0)
['M11'] Vessel departed at NLGOR 4 : 37 heading to NLRTM via route ('NLGOR', 'NLRTM', 0)
['M11'] Vessel departed at NLGOR 4 : 43 heading to NLRTM via route ('NLGOR', 'NLRTM', 0)
['M2'] Vessel departed at NLLEY 5 : 4 heading to NLAMS via route ('NLLEY', 'NLAMS', 0)
['M9'] Vessel departed at NLNIJ 5 : 43 heading to NLWAS via route ('NLNIJ', 'NLWAS', 0)
['M8'] Vessel departed at NLOOS 6 : 4 heading to NLTLB via route ('NLOOS', 'NLTLB', 0)
['M8'] Vessel departed at NLUTC 6 : 47 heading to NLRTM via route ('NLUTC', 'NLRTM', 0)
['M12'] Vessel departed at NLHAR 7 : 3 heading to NLWTE via route ('NLHAR', 'NLWTE', 0)
['M8'] Vessel departed at NLDOR 7 : 28 heading to NLRTM via route ('NLDOR', 'NLRTM', 0)
['M0'] Vessel departed at NLTNZ 7 : 37 heading to NLVLI via route ('NLTNZ', 'NLVLI', 0)
['M3'] Vessel departed at NLTLB

In [18]:
type_list

['M12',
 'M8',
 'BII-6b',
 'M10',
 'BIIa-1',
 'M9',
 'BII-6l',
 'C3b',
 'BII-4',
 'M7',
 'M6',
 'BIIL-1',
 'C3l',
 'M5',
 'M11',
 'BI',
 'M3',
 'M2',
 'M1',
 'BII-1',
 'BII-2b',
 'M4',
 'B03',
 'C4',
 'B04',
 'M0',
 'C2l',
 'BII-2L',
 'B02',
 'C1b',
 'C2b',
 'B01',
 'C1l']

In [19]:
def random_vessel_generator(df_prob, load=1):
    """ Function to generate random vessel data.
    Parameters
    ----------
    df_prob: pd.DataFrame
    This dataframe must contain historical travel data
    load: float
    This float is equal to the simulated occupation of the network and may be read as the part of the traffic that is
    electric.
    """
    df_prob = df_prob.loc[df_prob.trip_count != 0]
    df_prob.reset_index(inplace=True, drop=True)
    df_prob = df_prob.fillna(0)

    # create dict to store random prob based values later on
    main_dict = {i: [] for i in df_prob.columns}

    # copy origin and destination from original df
    main_dict['origin'] = []
    main_dict['destination'] = []
    main_dict['route_v'] = []
    main_dict['hour'] = []
    main_dict['trip_count'] = []

    # fill nan values and create type list
    type_list = list(df_prob.iloc[:, 4:-1])
    for i in range(24):
        df_temp = df_prob.loc[df_prob.hour == i]
        df_temp = df_temp.reset_index(drop=True)
        total = df_temp.trip_count.sum()
        prob_r = [i / total for i in df_temp.trip_count]
        to_pick_r = list(np.arange(0, len(df_temp)))
        count_r = total * 1
        # randomly draw ODs
        rand_routes = np.random.choice(a=to_pick_r, size=round(count_r), replace=True, p=prob_r)
        unique_r, counts_r = np.unique(rand_routes, return_counts=True)
        temp_type_dict_r = dict(zip(unique_r, counts_r))

        for pair in df_temp.index:
            # store trip count data previous draw
            if pair in temp_type_dict_r.keys():
                main_dict['trip_count'].append(temp_type_dict_r[pair])
                main_dict['origin'].append(df_temp.origin[pair])
                main_dict['destination'].append(df_temp.destination[pair])
                main_dict['route_v'].append(df_temp.route_v[pair])
                main_dict['hour'].append(i)
                # print(main_dict['trip_count'])
                # determine parameters to randomly draw ship types for each OD pair
                prob = list(df_temp.iloc[pair, 4:-1].values / df_temp.iloc[pair, 4:-1].sum())
                to_pick = type_list
                # goes wrong now because may be zero ...
                count = temp_type_dict_r[pair]

                # randomly draw ship types for each OD pair
                rand_vessels = np.random.choice(a=to_pick, size=round(count), replace=True, p=prob)
                unique, counts = np.unique(rand_vessels, return_counts=True)
                temp_type_dict = dict(zip(unique, counts))
                for key in type_list:
                    if key in temp_type_dict.keys():
                        main_dict[key].append(temp_type_dict[key])
                    else:
                        main_dict[key].append(0)
    break

    df_return = pd.DataFrame.from_dict(main_dict)

    return df_return

In [4]:
a1 = random_vessel_generator(df_ivs, 1)

In [93]:
a1

,origin,destination,trip_count,actual,hour,M12,M8,BII-6b,M10,BIIa-1,...,B04,M0,C2l,BII-2L,B02,C1b,C2b,B01,C1l,route_v
0,NLRTM,NLAMS,155,155,0,0,94,0,0,2,...,0,0,0,0,0,0,0,0,0,1
1,NLRTM,NLAMS,161,161,0,98,0,0,14,0,...,0,0,0,3,0,0,0,0,0,1
2,NLRTM,NLAMS,55,55,0,0,39,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,NLRTM,NLAMS,25,25,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,NLRTM,NLAMS,80,80,0,9,20,0,3,1,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2850,NLTNZ,NLVLI,5,5,23,0,5,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2851,NLTNZ,NLVLI,3,3,23,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2852,NLTNZ,NLVLI,19,19,23,0,16,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2853,NLTNZ,NLVLI,3,3,23,0,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
